# Introduction

The requirements for undergraduate CS certificate can be found at https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-ba/#requirementstext. 

# Mathematical Model

### Notations

$T$ is the max number of maximum number of semesters (e.g., $T = 6$ means that we only consider 3 years)

$C$ is the set of classes (e.g., $C = \{ \text{CS 200}, \text{CS 300}, \text{CS 524}, \cdots \}$)


### Decision variables

$x[t, c]$ is a Boolean variable to denote whether to take the class $c \in C$ at semester $t \in \{1, \cdots, T\}$. 

If a student takes CS 200 on his/her first semester, then $x[1, \text{CS 200}] = 1$

If a student takes CS 524 on the second Spring semester, then $x[3, \text{CS 524}]= 1$


### Prerequisite constraint

To take CS 300, the student need to take CS 200 in the previous semester, we can encode such prerequisite using the following constraint: 

$$x[t, \text{CS 300}] \le \sum_{t=1}^{T} x[t, \text{CS 200}] $$


CS 524 has the prerequisite similar to "(CS 200 or 300) and (MATH 340 or 341)", we can encode it with the constraint:

$$x[t, \text{CS 524}] \le \sum_{t=1}^{T} x[t, \text{CS 200}] + \sum_{t=1}^{T} x[t, \text{CS 300}] $$

$$x[t, \text{CS 524}] \le \sum_{t=1}^{T} x[t, \text{CS 340}] + \sum_{t=1}^{T} x[t, \text{CS 341}] $$


### Graduation requirement

In order to meet the graduation requirement, CS 300 must be taken, so we have the constraint:

$$\sum_{t=1}^{T} x[t, \text{CS 300}] \ge 1$$
    
There are other kinds of graduation requirements, such as taking $k$ coureses from a list of $C$, we can encode such requirements using the following constraint:

$$\sum_{t=1}^{T} \sum_{c \in C} x[t, c] \ge k$$
    
### Max credits

For undergraduate, we can take at most 18 credits in a given semester, so we need to ensure that

$$\sum_{c \in C} x[t, c] \cdot \text{credit}(c) \le 18, \quad \text{for all semester } t$$

### Objective 

A naive objective would be to minimize the sum of $x$: 

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c]$$

But this would lead to many equally good solutions. To avoid this, we add a weight $t$ to the class $c$ if its taken at semester $t$

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c] \cdot t$$

This avoid students procrastinating class to later semesters. 

# Data Preparation

We first load the json file obtained from the UW-Madison [Course Search & Enrollment website](enroll.wisc.edu), and extract all the CS courses.

In [112]:
using JSON

raw_data = JSON.parsefile("data/1214-spring-2021.json")["hits"]

cls_data = Dict(
    Symbol(c["courseDesignation"]) => 
    c for c in raw_data 
    if c["subject"]["shortDescription"] == "COMP SCI" && parse(Int, c["catalogSort"]) <= 700
)

cls = keys(cls_data)
n_cls = length(cls_data)

println("We have ", string(n_cls), " undergraduate courses in total: ", join(sort([String(e) for e in cls]), ","))

We have 57 undergraduate courses in total: COMP SCI 200,COMP SCI 220,COMP SCI 240,COMP SCI 252,COMP SCI 270,COMP SCI 298,COMP SCI 300,COMP SCI 304,COMP SCI 310,COMP SCI 319,COMP SCI 320,COMP SCI 352,COMP SCI 354,COMP SCI 368,COMP SCI 369,COMP SCI 400,COMP SCI 402,COMP SCI 407,COMP SCI 412,COMP SCI 425,COMP SCI 435,COMP SCI 475,COMP SCI 502,COMP SCI 506,COMP SCI 513,COMP SCI 514,COMP SCI 520,COMP SCI 524,COMP SCI 525,COMP SCI 532,COMP SCI 533,COMP SCI 534,COMP SCI 536,COMP SCI 537,COMP SCI 538,COMP SCI 539,COMP SCI 540,COMP SCI 542,COMP SCI 552,COMP SCI 558,COMP SCI 559,COMP SCI 564,COMP SCI 567,COMP SCI 570,COMP SCI 576,COMP SCI 577,COMP SCI 579,COMP SCI 611,COMP SCI 612,COMP SCI 638,COMP SCI 639,COMP SCI 640,COMP SCI 642,COMP SCI 679,COMP SCI 681,COMP SCI 682,COMP SCI 699


In [150]:
# show(IOContext(stdout, :limit => false), "text/plain", cls_data[Symbol("COMP SCI 524")]);
# 
join(keys(cls_data[Symbol("COMP SCI 524")]), " ")

# cls_data[Symbol("COMP SCI 524")]["creditRange"]

"honors allCrossListedSubjects breadths matched_queries termCode levels subjectAggregate courseId academicGroupCode gradingBasis advisoryPrerequisites ethnicStudies lettersAndScienceCredits approvedForTopics courseDesignationRaw openToFirstYear gradCourseWork catalogPrintFlag sustainability instructorProvidedContent courseRequirements subject repeatable fullCourseDesignationRaw typicallyOffered foreignLanguage enrollmentPrerequisites titleSuggest minimumCredits title lastUpdated workplaceExperience courseDesignation generalEd topics description lastTaught creditRange catalogSort currentlyTaught firstTaught catalogNumber maximumCredits fullCourseDesignation"

In [149]:
for (k, v) in sort(cls_data)
#     println(k, ": ", v["enrollmentPrerequisites"])
    println(k, ": ", v["creditRange"])
end

COMP SCI 200: 3
COMP SCI 220: 4
COMP SCI 240: 3
COMP SCI 252: 2
COMP SCI 270: 3
COMP SCI 298: 1-3
COMP SCI 300: 3
COMP SCI 304: 0-1
COMP SCI 310: 3
COMP SCI 319: 3
COMP SCI 320: 4
COMP SCI 352: 3
COMP SCI 354: 3
COMP SCI 368: 1
COMP SCI 369: 3
COMP SCI 400: 3
COMP SCI 402: 2
COMP SCI 407: 3
COMP SCI 412: 3
COMP SCI 425: 3
COMP SCI 435: 3
COMP SCI 475: 3
COMP SCI 502: 1
COMP SCI 506: 3
COMP SCI 513: 3
COMP SCI 514: 3
COMP SCI 520: 3
COMP SCI 524: 3
COMP SCI 525: 3
COMP SCI 532: 3
COMP SCI 533: 3
COMP SCI 534: 3
COMP SCI 536: 3
COMP SCI 537: 4
COMP SCI 538: 3
COMP SCI 539: 3
COMP SCI 540: 3
COMP SCI 542: 3
COMP SCI 552: 3
COMP SCI 558: 3
COMP SCI 559: 3
COMP SCI 564: 4
COMP SCI 567: 3
COMP SCI 570: 4
COMP SCI 576: 3
COMP SCI 577: 4
COMP SCI 579: 3
COMP SCI 611: 3
COMP SCI 612: 3
COMP SCI 638: 1-4
COMP SCI 639: 3-4
COMP SCI 640: 3
COMP SCI 642: 3
COMP SCI 679: 3
COMP SCI 681: 3
COMP SCI 682: 3
COMP SCI 699: 1-6


# Constructing the Optimization Problem

In [160]:
using JuMP, Cbc

max_credit = 18
n_sem = 4
T = 1:n_sem
C = cls

function credit(c)
    return cls_data[c]["minimumCredits"]
end

function cs_lst(arr...)
    res = [Symbol("COMP SCI ", c) for c in arr]
    return [c for c in res if c in C]
end

C_basic = cs_lst(240, 252, 300, 354, 400)
C_theory = cs_lst(577, 520)
C_xware = cs_lst(407, 506, 536, 538, 537, 552, 564, 640, 642)
C_app = cs_lst(412, 425, 513, 514, 524, 525, 534, 540, 545, 547, 559, 570)
C_elec = cs_lst(407, 412, 425, 435, 471, 475, 506, 513, 514, 520, 524, 525, 526, 532, 533, 534, 536, 537, 538, 539, 540, 545, 547, 552, 558, 559, 564, 567, 570, 576, 577, 579, 635, 640, 642, 679, 639)

# m = Model(Cbc.Optimizer)
m = Model(with_optimizer(Cbc.Optimizer, logLevel=0))

@variable(m, x[C, T], Bin)

# Take all from basic computer sciences
for c in C_basic
    @constraint(m, sum(x[c, t] for t in T) >= 1)
end

# Complete 1 for Theory of computer science
@constraint(m, sum(x[c, t] for t in T for c in C_theory) >= 1)

# Complete 2 for Software & Hardware
@constraint(m, sum(x[c, t] for t in T for c in C_xware) >= 2)

# Complete 1 for Applications
@constraint(m, sum(x[c, t] for t in T for c in C_app) >= 1)

# Complete 2 for Electives
@constraint(m, sum(x[c, t] for t in T for c in C_elec) >= 2)

# Max credit constraint
for t in T
    @constraint(m, sum(x[c, t] * credit(c) for c in C) <= max_credit)
end
    
@objective(m, Min, sum(x[c, t] * t for c in C, t in T))


optimize!(m)
println(termination_status(m))
x = value.(x)

for t in sems
    for c in cls
        if x[c, t] > 0
            println("Take course \"", c, "\" on semester ", t)
        end
    end
end

OPTIMAL
Take course "COMP SCI 400" on semester 1
Take course "COMP SCI 240" on semester 1
Take course "COMP SCI 252" on semester 1
Take course "COMP SCI 354" on semester 1
Take course "COMP SCI 640" on semester 1
Take course "COMP SCI 407" on semester 1
Take course "COMP SCI 570" on semester 2
Take course "COMP SCI 577" on semester 2
Take course "COMP SCI 300" on semester 2
